## Loading

In [143]:
import pandas as pd
import numpy as np

import os
import joblib

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.feature_extraction.text import CountVectorizer
import scipy
from scipy.sparse import hstack
from scipy.stats import uniform, randint



### Load caches 

In [5]:
df = pd.read_csv('./data/cleandata.csv')
label = pd.read_csv('./data/label.csv')

train_test_split = joblib.load("./cache/train_test_split.joblib")
meta_feat = joblib.load('./cache/meta_feat.joblib')
text_feat = joblib.load('./cache/text_feat.joblib')

In [55]:
features = np.concatenate((meta_feat, text_feat))
joblib.dump(features, './cache/features.joblib')

['./cache/features.joblib']

### Prepare data

In [29]:
df_train, df_test, val_train, val_test = train_test_split(df, label['up_votes'], test_size=.2, random_state=42)

In [31]:
def standardize(df, col):
    '''
    standardize a list of columns
    Arg: 
        df: pd.DataFrame
        col: list
    '''
    scaler = StandardScaler()
    return pd.DataFrame(scaler.fit_transform(df[col])).set_index(df.index)


In [32]:
df_train[['yearTonow', 'word_count']] = standardize(df_train, ['yearTonow', 'word_count'])
df_test[['yearTonow', 'word_count']] = standardize(df_test, ['yearTonow', 'word_count'])

/Users/jingciwang/anaconda3/envs/eluvio/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/jingciwang/anaconda3/envs/eluvio/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


### Combine Features

In [61]:
# get meta feature matrix
meta_train = scipy.sparse.csr_matrix(df_train[meta_feat])
meta_test = scipy.sparse.csr_matrix(df_test[meta_feat])

In [40]:
def get_text_matrix(df, col='title_clean', vocabulary=text_feat):

    vec = CountVectorizer(vocabulary=vocabulary)
    dtm = vec.fit_transform(df[col])
    return dtm

In [44]:
# get text feature matrix
text_train = get_text_matrix(df_train)
text_test = get_text_matrix(df_test)

In [67]:
# combine features matrix
m_train = hstack([meta_train, text_train])
m_test = hstack([meta_test, text_test])

## Model Selection

Consider that this is a regression problem, intuitively *Linear Regression* model should be sufficient to serve as benchmark model. Since it's not common for news title to have repeated words, most features are sort of binary variable. Therefore, I would expect tree-based models work for this problem.

**Models**:

 - Liear Regression
 - Random Forest
 - XGBoost

In [126]:
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, mean_absolute_error
from tabulate import tabulate

In [70]:
kfold = 3
cv = KFold(n_splits=kfold)

In [136]:
def predict_report(y_train, y_train_hat, y_test, y_test_hat):
    
    r2_train = r2_score(y_train_hat, y_train)
    r2_test = r2_score(y_test_hat, y_test)
    evs_train = explained_variance_score(y_train_hat, y_train)
    evs_test = explained_variance_score(y_test_hat, y_test)
    rmse_train = mean_squared_error(y_train_hat, y_train, squared=False)
    rmse_test = mean_squared_error(y_test_hat, y_test, squared=False)
    mae_train = mean_absolute_error(y_train_hat, y_train)
    mae_test = mean_absolute_error(y_test_hat, y_test)
    
    print(tabulate([['trainset', rmse_train, mae_train, r2_train, evs_train], \
                    ['testset', rmse_test, mae_test, r2_test, evs_test]], \
                   headers=['Set', 'RMSE', 'MAE', 'Explained_Variance', 'R2']))
    
    

### Linear Regression

In [117]:
lr_model = ElasticNet()
params = dict(alpha=uniform(), l1_ratio=uniform())
p_search = RandomizedSearchCV(lr_model, params, 
                              n_jobs=-1, random_state=42,
                              cv=cv, verbose=True,
                              scoring='neg_root_mean_squared_error')
p_search.fit(m_train, val_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   15.8s finished


RandomizedSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=ElasticNet(alpha=1.0, copy_X=True,
                                        fit_intercept=True, l1_ratio=0.5,
                                        max_iter=1000, normalize=False,
                                        positive=False, precompute=False,
                                        random_state=None, selection='cyclic',
                                        tol=0.0001, warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a37c5fe90>,
                                        'l1_ratio': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a37c5fe50>},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False,
                   scoring='neg_root

In [122]:
if not os.path.exists('./model'):
    os.mkdir('./model')
joblib.dump(p_search.best_estimator_, './model/ElasticNet.joblib') 

['./model/ElasticNet.joblib']

In [119]:
p_search.best_score_

-505.00782508319315

In [120]:
p_search.best_params_

{'alpha': 0.020584494295802447, 'l1_ratio': 0.9699098521619943}

In [128]:
y_train = p_search.best_estimator_.predict(m_train)
y_test = p_search.best_estimator_.predict(m_test)

In [137]:
predict_report(y_train, val_train, y_test, val_test)

Set          RMSE      MAE    Explained_Variance       R2
--------  -------  -------  --------------------  -------
trainset  503.366  168.107              0.13505   0.13505
testset   505.804  169.943              0.134155  0.13418


### Random Forest

In [173]:
rf_model = RandomForestRegressor(verbose=True)
params = {'max_depth': randint(2, 20),
 'n_estimators': randint(10, 50)}

p_search = RandomizedSearchCV(rf_model, params, 
                              n_jobs=-1, random_state=42,
                              cv=cv, verbose=True,
                              scoring='neg_root_mean_squared_error')
p_search.fit(m_train, val_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:   28.0s finished


RandomizedSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf...
                                

In [177]:
joblib.dump(p_search.best_estimator_, './model/RandomForest.joblib') 

['./model/RandomForest.joblib']

In [178]:
print(p_search.best_score_)
print(p_search.best_params_)

-506.96032672766023
{'max_depth': 9, 'n_estimators': 33}


In [179]:
y_train = p_search.best_estimator_.predict(m_train)
y_test = p_search.best_estimator_.predict(m_test)

predict_report(y_train, val_train, y_test, val_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Set          RMSE      MAE    Explained_Variance        R2
--------  -------  -------  --------------------  --------
trainset  498.686  160.974              0.151059  0.151061
testset   506.704  163.597              0.131071  0.131106


[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    0.1s finished


### XGBoost

In [184]:
xgb_model = GradientBoostingRegressor(verbose=True)
params = {'n_estimators': randint(10, 80), 
          'max_depth': randint(3, 10),
          'learning_rate': uniform(0.01, 0.1)}
p_search = RandomizedSearchCV(xgb_model, params, 
                              n_jobs=-1, random_state=42,
                              cv=cv, verbose=True,
                              scoring='neg_root_mean_squared_error')
p_search.fit(m_train, val_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.2min finished


      Iter       Train Loss   Remaining Time 
         1      287810.7062            1.14m
         2      283152.3651            1.08m
         3      279042.1316            1.06m
         4      275395.6226            1.02m
         5      272125.5904            1.00m
         6      269068.0243           58.96s
         7      266299.2200           57.55s
         8      263821.2993           56.18s
         9      261598.0747           54.31s
        10      259613.7543           52.47s
        20      247084.9838           37.94s
        30      240563.3046           25.06s
        40      237366.8802           12.79s
        50      234471.0133            1.14s


RandomizedSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,...
                   param_distributions={'l

In [185]:
joblib.dump(p_search.best_estimator_, './model/XGBoost.joblib') 

['./model/XGBoost.joblib']

In [186]:
print(p_search.best_score_)
print(p_search.best_params_)

-506.63399713388054
{'learning_rate': 0.06247746602583892, 'max_depth': 9, 'n_estimators': 51}


In [187]:
y_train = p_search.best_estimator_.predict(m_train)
y_test = p_search.best_estimator_.predict(m_test)

predict_report(y_train, val_train, y_test, val_test)

Set          RMSE      MAE    Explained_Variance        R2
--------  -------  -------  --------------------  --------
trainset  483.941  159.032              0.200518  0.200518
testset   505.191  163.7                0.136253  0.136286


## Summary

Overall, *Tree-based models* have much better performance on train set than linear regression. But after several rounds of hyper-parameter tuning, both *Random Forest* and *XGBoost* have inclination to overfitting and parameters like *tree-depth* and *number of estimator* have to be set relatively small. *Linear Regression* have least difference between error on train set and that on test, after tuning the hyper-parameter, larger ratio of l1 regularizer would give better performance.

All three models have similar outputs on test set, but that of *XGBoost* is sightly better than the other two models. Considering the scale of the dataset, the features selected are capable of predicting up_votes to some extent.